#Criar um script que faça:  
#Pegue as nfe em xml do email  
#Extrai os dados necessarios   
#Tratar dados   
#Remover materiais pesado E outras notas 
-TABUAS CABOQUINHO
-CIMENTRO VOTORAN
-Magalu 
#Aplicador formula de taxas  
#Inserir no template dos marketplaces
  

In [6]:
import os
import shutil
import xml.etree.ElementTree as ET
import pandas as pd
from imap_tools import MailBox, AND
from config import pwd, user
from datetime import date
import openpyxl

In [ ]:
#Baixa os anexos XML dos emails
save_folder = "notas/nfes/"
os.makedirs(save_folder, exist_ok=True)

with MailBox("imap.gmail.com").login(user, pwd, initial_folder="INBOX", ) as mailbox:
    list_mail = mailbox.fetch(criteria=AND(date=date.today())

    )
    for email in list_mail:
        for anexo in email.attachments:
            if  anexo.filename.lower().endswith(".xml"):
                file_path = os.path.join(save_folder, anexo.filename)
                if not os.path.exists(file_path):   
                    with open(file_path, "wb") as f:
                        f.write(anexo.payload)

In [8]:
#Salva os XMLs em pastas por Emitente e extrai os produtos para um Excel

pasta_origem = "notas/nfes"
todos_produtos = []


def extrai_dados(caminho_arquivo):
    produtos = []
    tree = ET.parse(caminho_arquivo)
    root = tree.getroot()
    ns = {"ns": "http://www.portalfiscal.inf.br/nfe"}
    
    natOp = root.find(".//ns:ide/ns:natOp", ns)
    if natOp is not None and natOp.text.strip().upper() == 'BONIFICACAO, DOACAO OU BRINDE':
        return [] 
    
    for det in root.findall(".//ns:det", ns):   
        produto = {}
        produto ['Codigo Produto'] = det.find("./ns:prod/ns:cProd", ns).text
        produto ['Descrição'] = det.find("./ns:prod/ns:xProd", ns).text
        produto ['Valor_unitário'] = det.find("./ns:prod/ns:vUnCom", ns).text
        produto ['Código de Barras'] = det.find("./ns:prod/ns:cEAN", ns).text 
        produto ['Sku'] = produto['Código de Barras']
        if produto['Sku'] == 'SEM GTIN':
            produto['Sku'] = produto['Descrição']
        produto ['Fornecedor'] = root.find(".//ns:emit/ns:xNome", ns).text
        produtos.append(produto)
    return produtos
        


# Percorre todos os arquivos da pasta
for arquivo in os.listdir(pasta_origem):
    if arquivo.lower().endswith(".xml"):
        caminho_arquivo = os.path.join(pasta_origem, arquivo)

        try:
            # Abre e parseia o XML
            tree = ET.parse(caminho_arquivo)
            root = tree.getroot()

            # Define namespace
            ns = {"ns": "http://www.portalfiscal.inf.br/nfe"}

            # Busca o CNPJ do emitente
            cnpj_emitente = root.find(".//ns:emit/ns:CNPJ", ns)
            cnpj_emitente
            if cnpj_emitente is None:
                print(f"⚠️ CNPJ não encontrado em {arquivo}")
                continue

            cnpj_emitente = cnpj_emitente.text

            produtos = extrai_dados(caminho_arquivo)
            todos_produtos.extend(produtos)              


            emitente = root.find(".//ns:emit/ns:xNome", ns)
            pasta_destino = os.path.join("notas/", emitente.text)

            # Cria a pasta se não existir
            os.makedirs(pasta_destino, exist_ok=True)

            # Copia o arquivo para a pasta
            shutil.copy(caminho_arquivo, pasta_destino)

        except Exception as e:
            print(f" {arquivo}: {e}")


df = pd.DataFrame(todos_produtos)


final = df.drop_duplicates(subset=['Sku'], keep='first')

final


final.to_excel("produtos_extraidos.xlsx", index=False)
sku = df.query('Sku == Descrição')
sku.to_excel("sem_gtin.xlsx", index=False)

In [ ]:
#TESTANDO WEB BeautifulSoup para buscar gtin em falta

In [16]:
sem_gtin = pd.read_excel('sem_gtin.xlsx')

sem_gtin['Fornecedor']

import requests


for index, produtos_sem_gtin in sem_gtin.iterrows():
    if produtos_sem_gtin['Fornecedor'] == 'CONSTRUDIGI DISTRIBUIDORA DE MATERIAIS PARA CONSTRUCAO LTDA':
        url = f'https://www.construdigi.com.br/produto/{produtos_sem_gtin["Codigo Produto"]}/{produtos_sem_gtin["Descrição"]}'
   


import requests
from bs4 import BeautifulSoup

url = "https://www.construdigi.com.br/produto/104217/multilit-joelho-esg-75x90"
headers = {"User-Agent": "Mozilla/5.0"}

response = requests.get(url, headers=headers)
if response.status_code != 200:
    raise Exception(f"Erro ao acessar a página: {response.status_code}")

soup = BeautifulSoup(response.text, "html.parser")

# Procurar celula que contenha o texto 'Peso:' ou similar
td_peso = soup.find("td", string=lambda s: s and "Peso" in s)
if td_peso:
    valor_peso = td_peso.find_next_sibling("td").get_text(strip=True)
    print("Peso encontrado:", valor_peso)
else:
    print("Campo de peso não foi encontrado na página.")


Campo de peso não foi encontrado na página.


In [20]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time

options = Options()
options.add_argument("--headless")  # modo sem interface gráfica
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

driver.get("https://www.construdigi.com.br/produto/104217/multilit-joelho-esg-75x90")
time.sleep(2)

peso_label = driver.find_element(By.XPATH, "//td[contains(text(), 'Peso Bruto')]")
peso_valor = peso_label.find_element(By.XPATH, "following-sibling::td").text.strip()

print("Peso bruto encontrado:", peso_valor)

driver.quit()


Peso bruto encontrado: 0.15600000
